In [7]:
# Install pyarrow to support parquet files
!pip install pyarrow

Defaulting to user installation because normal site-packages is not writeable


In [2]:
import pandas as pd

In [3]:
df = pd.read_parquet("documents/yellow_tripdata_2023-02.parquet", engine="pyarrow")

In [4]:
df.to_csv("yellow_tripdata_2023-02.csv")

In [41]:
df.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,trip_id
0,1,2023-02-01 00:32:53,2023-02-01 00:34:34,2,0.30,1,N,142,163,2,4.4,3.50,0.5,0.0,0.0,1.0,9.40,2.5,0.00,0
1,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1,0.00,1,N,71,71,4,-3.0,-1.00,-0.5,0.0,0.0,-1.0,-5.50,0.0,0.00,1
2,2,2023-02-01 00:35:16,2023-02-01 00:35:30,1,0.00,1,N,71,71,4,3.0,1.00,0.5,0.0,0.0,1.0,5.50,0.0,0.00,2
3,1,2023-02-01 00:29:33,2023-02-01 01:01:38,0,18.80,1,N,132,26,1,70.9,2.25,0.5,0.0,0.0,1.0,74.65,0.0,1.25,3
4,2,2023-02-01 00:12:28,2023-02-01 00:25:46,1,3.22,1,N,161,145,1,17.0,1.00,0.5,3.3,0.0,1.0,25.30,2.5,0.00,4


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2913955 entries, 0 to 2913954
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [12]:
# We are going to modify the columns dtype to int
# Ee  use the fillna() method to replace missing values with 0, 
# And then the replace() method to replace infinite values with 0. 
# Finally, we use the astype() method to convert the columns to the int32 data type.
import numpy as np

df["passenger_count"] = df["passenger_count"].fillna(0).replace([np.inf, -np.inf], 0).astype("int32")
df["RatecodeID"] = df["RatecodeID"].fillna(0).replace([np.inf, -np.inf], 0).astype("int32")


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2913955 entries, 0 to 2913954
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int32         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        int32         
 4   trip_distance          float64       
 5   RatecodeID             int32         
 6   store_and_fwd_flag     object        
 7   PULocationID           int32         
 8   DOLocationID           int32         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  Airport_fee           

In [14]:
# This code drops any duplicate rows in the DataFrame 
# and adds a new column 'trip_id' with unique integer values starting from 0 to the length of the DataFrame.

df = df.drop_duplicates().reset_index(drop=True)
df["trip_id"] = df.index

In [24]:
# Separate datetime values into different columns

datetime_dim = df[["tpep_pickup_datetime","tpep_dropoff_datetime"]].reset_index(drop=True)
datetime_dim["pick_hour"] = datetime_dim["tpep_pickup_datetime"].dt.hour
datetime_dim["pick_day"] = datetime_dim["tpep_pickup_datetime"].dt.day
datetime_dim["pick_month"] = datetime_dim["tpep_pickup_datetime"].dt.month
datetime_dim["pick_year"] = datetime_dim["tpep_pickup_datetime"].dt.year
datetime_dim["pick_weekday"] = datetime_dim["tpep_pickup_datetime"].dt.weekday

datetime_dim["drop_hour"] = datetime_dim["tpep_dropoff_datetime"].dt.hour
datetime_dim["drop_day"] = datetime_dim["tpep_dropoff_datetime"].dt.day
datetime_dim["drop_month"] = datetime_dim["tpep_dropoff_datetime"].dt.month
datetime_dim["drop_year"] = datetime_dim["tpep_dropoff_datetime"].dt.year
datetime_dim["drop_weekday"] = datetime_dim["tpep_dropoff_datetime"].dt.weekday

In [25]:
datetime_dim

,tpep_pickup_datetime,tpep_dropoff_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,2023-02-01 00:32:53,2023-02-01 00:34:34,0,1,2,2023,2,0,1,2,2023,2
1,2023-02-01 00:35:16,2023-02-01 00:35:30,0,1,2,2023,2,0,1,2,2023,2
2,2023-02-01 00:35:16,2023-02-01 00:35:30,0,1,2,2023,2,0,1,2,2023,2
3,2023-02-01 00:29:33,2023-02-01 01:01:38,0,1,2,2023,2,1,1,2,2023,2
4,2023-02-01 00:12:28,2023-02-01 00:25:46,0,1,2,2023,2,0,1,2,2023,2
...,...,...,...,...,...,...,...,...,...,...,...,...
2913950,2023-02-28 23:46:00,2023-03-01 00:05:00,23,28,2,2023,1,0,1,3,2023,2
2913951,2023-02-28 23:26:02,2023-02-28 23:37:10,23,28,2,2023,1,23,28,2,2023,1
2913952,2023-02-28 23:24:00,2023-02-28 23:38:00,23,28,2,2023,1,23,28,2,2023,1
2913953,2023-02-28 23:03:00,2023-02-28 23:10:00,23,28,2,2023,1,23,28,2,2023,1


In [26]:
# Now we add the index and primary key and reorder the columns

datetime_dim["datetime_id"] = datetime_dim.index
datetime_dim = datetime_dim[["datetime_id", "tpep_pickup_datetime", "pick_hour", "pick_day", "pick_month", "pick_year", "pick_weekday",
                             "tpep_dropoff_datetime", "drop_hour", "drop_day", "drop_month", "drop_year", "drop_weekday"]]

In [27]:
datetime_dim

,datetime_id,tpep_pickup_datetime,pick_hour,pick_day,pick_month,pick_year,pick_weekday,tpep_dropoff_datetime,drop_hour,drop_day,drop_month,drop_year,drop_weekday
0,0,2023-02-01 00:32:53,0,1,2,2023,2,2023-02-01 00:34:34,0,1,2,2023,2
1,1,2023-02-01 00:35:16,0,1,2,2023,2,2023-02-01 00:35:30,0,1,2,2023,2
2,2,2023-02-01 00:35:16,0,1,2,2023,2,2023-02-01 00:35:30,0,1,2,2023,2
3,3,2023-02-01 00:29:33,0,1,2,2023,2,2023-02-01 01:01:38,1,1,2,2023,2
4,4,2023-02-01 00:12:28,0,1,2,2023,2,2023-02-01 00:25:46,0,1,2,2023,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913950,2913950,2023-02-28 23:46:00,23,28,2,2023,1,2023-03-01 00:05:00,0,1,3,2023,2
2913951,2913951,2023-02-28 23:26:02,23,28,2,2023,1,2023-02-28 23:37:10,23,28,2,2023,1
2913952,2913952,2023-02-28 23:24:00,23,28,2,2023,1,2023-02-28 23:38:00,23,28,2,2023,1
2913953,2913953,2023-02-28 23:03:00,23,28,2,2023,1,2023-02-28 23:10:00,23,28,2,2023,1


In [42]:
# And we all the same with all the dim tables

passenger_count_dim = df[["passenger_count"]].reset_index(drop=True)
passenger_count_dim["passenger_count_id"] = passenger_count_dim.index
passenger_count_dim = passenger_count_dim[["passenger_count_id","passenger_count"]]

trip_distance_dim = df[["trip_distance"]].reset_index(drop=True)
trip_distance_dim["trip_distance_id"] = trip_distance_dim.index
trip_distance_dim = trip_distance_dim[["trip_distance_id","trip_distance"]]

pickup_location_dim = df[["PULocationID"]].reset_index(drop=True)
pickup_location_dim["pickup_location_id"] = pickup_location_dim.index
pickup_location_dim = pickup_location_dim[["pickup_location_id","PULocationID"]]

dropoff_location_dim = df[["DOLocationID"]].reset_index(drop=True)
dropoff_location_dim["dropoff_location_id"] = dropoff_location_dim.index
dropoff_location_dim = dropoff_location_dim[["dropoff_location_id","DOLocationID"]]

In [40]:
passenger_count_dim.head()

,passenger_count_id,passenger_count
0,0,2
1,1,1
2,2,1
3,3,0
4,4,1


In [33]:
trip_distance_dim

,trip_distance_id,trip_distance
0,0,0.30
1,1,0.00
2,2,0.00
3,3,18.80
4,4,3.22
...,...,...
2913950,2913950,4.65
2913951,2913951,2.47
2913952,2913952,3.49
2913953,2913953,2.13


In [43]:
pickup_location_dim

,pickup_location_id,PULocationID
0,0,142
1,1,71
2,2,71
3,3,132
4,4,161
...,...,...
2913950,2913950,249
2913951,2913951,186
2913952,2913952,158
2913953,2913953,79


In [44]:
dropoff_location_dim

,dropoff_location_id,DOLocationID
0,0,163
1,1,71
2,2,71
3,3,26
4,4,145
...,...,...
2913950,2913950,140
2913951,2913951,79
2913952,2913952,143
2913953,2913953,162


In [34]:
# Here we will create a dictionary because we want to include the rate code name column into the Dim table
# We use the map function to link to their corresponding string labels

rate_code_type = {
    1:"Standard rate",
    2:"JFK",
    3:"Newark",
    4:"Nassau or Westchester",
    5:"Negotiated fare",
    6:"Group ride"
}

rate_code_dim = df[["RatecodeID"]].reset_index(drop=True)
rate_code_dim["rate_code_id"] = rate_code_dim.index
rate_code_dim["rate_code_name"] = rate_code_dim["RatecodeID"].map(rate_code_type)
rate_code_dim = rate_code_dim[["rate_code_id","RatecodeID","rate_code_name"]]

In [38]:
rate_code_dim.head()

,rate_code_id,RatecodeID,rate_code_name
0,0,1,Standard rate
1,1,1,Standard rate
2,2,1,Standard rate
3,3,1,Standard rate
4,4,1,Standard rate


In [36]:

payment_type_name = {
    1:"Credit card",
    2:"Cash",
    3:"No charge",
    4:"Dispute",
    5:"Unknown",
    6:"Voided trip"
}
payment_type_dim = df[["payment_type"]].reset_index(drop=True)
payment_type_dim["payment_type_id"] = payment_type_dim.index
payment_type_dim["payment_type_name"] = payment_type_dim["payment_type"].map(payment_type_name)
payment_type_dim = payment_type_dim[["payment_type_id","payment_type","payment_type_name"]]

In [39]:
payment_type_dim.head()

,payment_type_id,payment_type,payment_type_name
0,0,2,Cash
1,1,4,Dispute
2,2,4,Dispute
3,3,1,Credit card
4,4,1,Credit card


In [45]:
# Now we have to join all dimensional tables into the fact table

fact_table = df.merge(passenger_count_dim, left_on="trip_id", right_on="passenger_count_id") \
                .merge(trip_distance_dim, left_on="trip_id", right_on="trip_distance_id") \
                .merge(rate_code_dim, left_on="trip_id", right_on="rate_code_id") \
                .merge(pickup_location_dim, left_on="trip_id", right_on="pickup_location_id") \
                .merge(dropoff_location_dim, left_on="trip_id", right_on="dropoff_location_id") \
                .merge(datetime_dim, left_on="trip_id", right_on="datetime_id") \
                .merge(payment_type_dim, left_on="trip_id", right_on="payment_type_id") \
                [["trip_id","VendorID", "datetime_id", "passenger_count_id",
                "trip_distance_id", "rate_code_id", "store_and_fwd_flag", "pickup_location_id", "dropoff_location_id",
                "payment_type_id", "fare_amount", "extra", "mta_tax", "tip_amount", "tolls_amount",
                "improvement_surcharge", "total_amount"]]

In [46]:
fact_table.columns

Index(['trip_id', 'VendorID', 'datetime_id', 'passenger_count_id',
       'trip_distance_id', 'rate_code_id', 'store_and_fwd_flag',
       'pickup_location_id', 'dropoff_location_id', 'payment_type_id',
       'fare_amount', 'extra', 'mta_tax', 'tip_amount', 'tolls_amount',
       'improvement_surcharge', 'total_amount'],
      dtype='object')

In [47]:
fact_table

,trip_id,VendorID,datetime_id,passenger_count_id,trip_distance_id,rate_code_id,store_and_fwd_flag,pickup_location_id,dropoff_location_id,payment_type_id,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount
0,0,1,0,0,0,0,N,0,0,0,4.40,3.50,0.5,0.00,0.0,1.0,9.40
1,1,2,1,1,1,1,N,1,1,1,-3.00,-1.00,-0.5,0.00,0.0,-1.0,-5.50
2,2,2,2,2,2,2,N,2,2,2,3.00,1.00,0.5,0.00,0.0,1.0,5.50
3,3,1,3,3,3,3,N,3,3,3,70.90,2.25,0.5,0.00,0.0,1.0,74.65
4,4,2,4,4,4,4,N,4,4,4,17.00,1.00,0.5,3.30,0.0,1.0,25.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2913950,2913950,2,2913950,2913950,2913950,2913950,None,2913950,2913950,2913950,20.22,0.00,0.5,4.84,0.0,1.0,29.06
2913951,2913951,2,2913951,2913951,2913951,2913951,None,2913951,2913951,2913951,13.66,0.00,0.5,2.65,0.0,1.0,20.31
2913952,2913952,2,2913952,2913952,2913952,2913952,None,2913952,2913952,2913952,17.64,0.00,0.5,0.00,0.0,1.0,21.64
2913953,2913953,2,2913953,2913953,2913953,2913953,None,2913953,2913953,2913953,13.56,0.00,0.5,2.63,0.0,1.0,20.19
